In [2]:
#1 Racer
#Imports
import pygame, sys

from pygame.locals import *
import random, time
 
#Initialzing 
pygame.init()
pygame.mixer.init()

 
#Setting up FPS 
FPS = 60
FramePerSec = pygame.time.Clock()
 
#Creating colors
BLUE  = (0, 0, 255)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
 
#Other Variables for use in the program
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0 #for counting score
COIN_SPEED = 5
count = 0 #for counting numbers of coins
 
#Setting up Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)
 
background = pygame.image.load("AnimatedStreet.png")
pygame.mixer.music.load("background.wav")
pygame.mixer.music.play(-1) 

 
#Create a white screen 
DISPLAYSURF = pygame.display.set_mode((400,600))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("Game")
 
class Enemy(pygame.sprite.Sprite):
      def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH-40), 0)  
 
      def move(self):
        global SCORE
        self.rect.move_ip(0,SPEED)
        if (self.rect.top > 600):
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)
 
 
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__() 
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)
        
    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if pressed_keys[K_UP]:
            self.rect.move_ip(0, -5)
        if pressed_keys[K_DOWN]:
            self.rect.move_ip(0,5)
         
        if self.rect.left > 0:
              if pressed_keys[K_LEFT]:
                  self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH:        
              if pressed_keys[K_RIGHT]:
                  self.rect.move_ip(5, 0)
#creating coins
class Coin:
    def __init__(self):
        self.radius = 10
        self.x = random.randint(50, SCREEN_WIDTH - 50)
        self.y = -50
        self.speed = COIN_SPEED
    def move(self):
        self.y+=self.speed
        if self.y > SCREEN_HEIGHT:
            self.reset_position()
    def reset_position(self):
        self.y = -50
    def draw(self, surface):
        pygame.draw.circle(surface, (255,255,0), (self.x, self.y), self.radius)




#Setting up Sprites        
P1 = Player()
E1 = Enemy()
C1 = Coin()

 
#Creating Sprites Groups
enemies = pygame.sprite.Group()
enemies.add(E1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)

 
#Adding a new User event 
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)
 
#Game Loop
while True:
       
    #Cycles through all events occurring  
    for event in pygame.event.get():
        if event.type == INC_SPEED:
              SPEED += 0.5     
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
 
    DISPLAYSURF.blit(background, (0,0))
    scores = font_small.render(str(SCORE), True, BLACK)
    DISPLAYSURF.blit(scores, (10,10))
    counts = font_small.render(f"Coins: {str(count)}", True, BLACK)
    DISPLAYSURF.blit(counts, (SCREEN_WIDTH-110,10 )) #the place of the coin amounts
    

    C1.move()
    C1.draw(DISPLAYSURF)
    
 
    #Moves and Re-draws all Sprites
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        if hasattr(entity, "move"):  # Проверяем, есть ли метод move
            entity.move()

    
   #To be run if collision occurs between Player and Enemy
    if pygame.sprite.spritecollideany(P1, enemies):
          pygame.mixer.Sound('crash.wav').play()
          time.sleep(0.5)
                    
          DISPLAYSURF.fill(RED)
          DISPLAYSURF.blit(game_over, (30,250))
           
          pygame.display.update()
          for entity in all_sprites:
                entity.kill() 
          time.sleep(2)
          pygame.quit()
          sys.exit() 
    
    if P1.rect.colliderect(pygame.Rect(C1.x - C1.radius, C1.y - C1.radius, C1.radius * 2, C1.radius * 2)):
        count += 1  # We give you more points for a coin
        C1.reset_position() 
    
           
    pygame.display.update()
    FramePerSec.tick(FPS)

SystemExit: 

In [3]:
#2 snake
import pygame
import random

pygame.init()

w, h = 800, 800
BLOCK_SIZE = 50
font = pygame.font.Font(None, 100)
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Snake game")
clock = pygame.time.Clock()
run = True

def drawGrid():#creating cells
    for x in range(0, w, BLOCK_SIZE):
        for y in range(0, h, BLOCK_SIZE):
            rect = pygame.Rect(x, y, BLOCK_SIZE, BLOCK_SIZE)
            pygame.draw.rect(screen, (0, 0, 0), rect, 1) 

class Snake: #creating snake
    def __init__(self):
        self.x, self.y = BLOCK_SIZE, BLOCK_SIZE
        self.xdir = 1
        self.ydir = 0
        self.head = pygame.Rect(self.x, self.y, BLOCK_SIZE, BLOCK_SIZE)
        self.body = [pygame.Rect(self.x - BLOCK_SIZE, self.y, BLOCK_SIZE, BLOCK_SIZE)]
        self.dead = False

    def update(self): 
        global apple, start, lev
        for square in self.body:
            if square.x == self.head.x and square.y == self.head.y:
                self.dead = True

        if self.head.x < 0 or self.head.x >= w or self.head.y < 0 or self.head.y >= h:
            self.dead = True

        if self.dead:
            self.__init__()
            apple = Apple()
            start = 0  
            lev = 1  

        self.body.append(self.head.copy())  
        self.head.x += self.xdir * BLOCK_SIZE
        self.head.y += self.ydir * BLOCK_SIZE
        self.body.pop(0)  

class Apple: #creating food
    def __init__(self): #random appearance of food
        self.x = random.randint(0, (w // BLOCK_SIZE) - 1) * BLOCK_SIZE 
        self.y = random.randint(0, (h // BLOCK_SIZE) - 1) * BLOCK_SIZE
        self.rect = pygame.Rect(self.x, self.y, BLOCK_SIZE, BLOCK_SIZE)

    def update(self):
        pygame.draw.rect(screen, (255, 0, 0), self.rect)

start = 0  #count score
lev = 1 #count level
speed = 10
snake = Snake()
apple = Apple()

while run:
    screen.fill((255, 255, 255))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_DOWN and snake.ydir == 0:
                snake.ydir = 1
                snake.xdir = 0   
            elif event.key == pygame.K_UP and snake.ydir == 0:
                snake.ydir = -1
                snake.xdir = 0
            elif event.key == pygame.K_RIGHT and snake.xdir == 0:
                snake.ydir = 0
                snake.xdir = 1
            elif event.key == pygame.K_LEFT and snake.xdir == 0:
                snake.ydir = 0
                snake.xdir = -1

    snake.update()
    drawGrid()
    apple.update()

    score = font.render(f"{start}", True, (0, 0, 0))
    score_pos = score.get_rect(center=(w / 2, h / 20))       #location of score
    screen.blit(score, score_pos)

    level = font.render(f"{lev} level", True, (0, 0, 0))
    level_pos = level.get_rect(center=(w / 2, h / 8))      #location of level
    screen.blit(level, level_pos)

    pygame.draw.rect(screen, (255, 255, 0), snake.head)
    
    for sq in snake.body:
        pygame.draw.rect(screen, (0, 255, 0), sq)

    
    if snake.head.x == apple.x and snake.head.y == apple.y: #snake lengthening
        last_part = snake.body[-1]
        snake.body.append(pygame.Rect(last_part.x, last_part.y, BLOCK_SIZE, BLOCK_SIZE))
        apple = Apple()
        start += 1  
        if start % 4 == 0:
            lev += 1
            speed+=2

    pygame.display.update()
    clock.tick(speed)

pygame.quit()

In [ ]:
#3 Paint
import pygame

def main():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    clock = pygame.time.Clock()
    
    radius = 15
    mode = 'blue'
    points = []

    regims = 'rect'
    rect_start = None
    circle_start = None

    rectangles = []
    circles = []

    
    #function for erase
    def erase_objects(position, radius):
        nonlocal points, rectangles, circles
    
        points = [p for p in points if (p[0] - position[0]) ** 2 + (p[1] - position[1]) ** 2 > radius ** 2]
        
       
        rectangles = [r for r in rectangles if not (r[0] < position[0] < r[0] + r[2] and r[1] < position[1] < r[1] + r[3])]
        
        
        circles = [c for c in circles if (c[0][0] - position[0]) ** 2 + (c[0][1] - position[1]) ** 2 > radius ** 2]

    while True:
        
        pressed = pygame.key.get_pressed()
        
        alt_held = pressed[pygame.K_LALT] or pressed[pygame.K_RALT]
        ctrl_held = pressed[pygame.K_LCTRL] or pressed[pygame.K_RCTRL]
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w and ctrl_held:
                    return
                if event.key == pygame.K_F4 and alt_held:
                    return
                if event.key == pygame.K_ESCAPE:
                    return
            
                if event.key == pygame.K_r:
                    mode = 'red'
                elif event.key == pygame.K_g:
                    mode = 'green'
                elif event.key == pygame.K_b:
                    mode = 'blue'
                elif event.key == pygame.K_y:
                    mode = 'yellow'
                elif event.key == pygame.K_p:
                    mode = 'purple'
                elif event.key == pygame.K_w:
                    mode = 'white'
        
                # regimes
                elif event.key == pygame.K_t:
                    regims = 'rect'
                elif event.key == pygame.K_c:
                    regims = 'circle'
                elif event.key == pygame.K_e:
                    regims = 'eraser'
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                if regims == 'rect':
                    rect_start = event.pos
                elif regims == 'circle':
                    circle_start = event.pos
            
            if event.type == pygame.MOUSEBUTTONUP:
                if regims == 'rect' and rect_start:
                    end_pos = event.pos
                    rect_x = min(rect_start[0], end_pos[0])
                    rect_y = min(rect_start[1], end_pos[1])
                    rect_width = abs(end_pos[0] - rect_start[0])
                    rect_height = abs(end_pos[1] - rect_start[1])
                    rectangles.append((rect_x, rect_y, rect_width, rect_height, mode))
                    rect_start = None
                elif regims == 'circle' and circle_start:
                    end_pos = event.pos
                    radius = int(((end_pos[0] - circle_start[0]) ** 2 + (end_pos[1] - circle_start[1]) ** 2) ** 0.5)
                    circles.append((circle_start, radius, mode))
                    circle_start = None

            if event.type == pygame.MOUSEMOTION:
                position = event.pos
                if regims == 'eraser':  # If the eraser is enabled
                    erase_objects(position, radius)  # Erasing objects
                else:
                    points.append(position)
                    points = points[-256:]

                
        screen.fill((0, 0, 0))

        # drawing rectangle
        for rect in rectangles:
            pygame.draw.rect(screen, getColor(rect[4]), (rect[0], rect[1], rect[2], rect[3]), 2)

        # drawing circle
        for circle in circles:
            pygame.draw.circle(screen, getColor(circle[2]), circle[0], circle[1], 2)
        
        # drawing lines
        i = 0
        while i < len(points) - 1:
            drawLineBetween(screen, i, points[i], points[i + 1], radius, mode)
            i += 1

        # Displaying the current rectangle when drawing
        if rect_start and pygame.mouse.get_pressed()[0]:
            end_pos = pygame.mouse.get_pos()
            rect_x = min(rect_start[0], end_pos[0])
            rect_y = min(rect_start[1], end_pos[1])
            rect_width = abs(end_pos[0] - rect_start[0])
            rect_height = abs(end_pos[1] - rect_start[1])
            pygame.draw.rect(screen, getColor(mode), (rect_x, rect_y, rect_width, rect_height), 2)

        # Displaying the current circle when drawing
        if circle_start and pygame.mouse.get_pressed()[0]:
            end_pos = pygame.mouse.get_pos()
            radius = int(((end_pos[0] - circle_start[0]) ** 2 + (end_pos[1] - circle_start[1]) ** 2) ** 0.5)
            pygame.draw.circle(screen, getColor(mode), circle_start, radius, 2)

        # Eraser display
        if regims == 'eraser':
            pygame.draw.circle(screen, (255, 255, 255), pygame.mouse.get_pos(), radius, 1)  

        pygame.display.flip()
        
        clock.tick(60)

def drawLineBetween(screen, index, start, end, width, color_mode):
    c1 = max(0, min(255, 2 * index - 256))
    c2 = max(0, min(255, 2 * index))
    
    if color_mode == 'blue':
        color = (c1, c1, c2)
    elif color_mode == 'red':
        color = (c2, c1, c1)
    elif color_mode == 'green':
        color = (c1, c2, c1)
    elif color_mode == 'yellow':
        color = (255, 255, 0)
    elif color_mode == 'purple':
        color = (128,0,128)
    elif color_mode == 'orange':
        color = (255,165,0)
    elif color_mode == 'white':
        color = (255,255,255)
    
    dx = start[0] - end[0]
    dy = start[1] - end[1]
    iterations = max(abs(dx), abs(dy))
    
    for i in range(iterations):
        progress = 1.0 * i / iterations
        aprogress = 1 - progress
        x = int(aprogress * start[0] + progress * end[0])
        y = int(aprogress * start[1] + progress * end[1])
        pygame.draw.circle(screen, color, (x, y), width)

# colors

def getColor(mode):
    colors = {
        'blue': (0, 0, 255),
        'red': (255, 0, 0),
        'green': (0, 255, 0),
        'yellow': (255, 255, 0),
        'purple': (128, 0, 128),
        'orange': (255, 165, 0),
        'white': (255, 255, 255),
    }
    return colors.get(mode, (255, 255, 255))

main()


: 